# Checking enviroment states and action sets

In [1]:
import sys
import os.path
sys.path.append("..")
from PPO_Agent_env_model_v2 import * #PPO_Agent_v2  PPO_Agent_with_Guided_AC
from ENV_DETAILS import *
from RUN_TENSORBOARD import *

events_folder = "./logs_hyper"
main("./logs_hyper")

2024-11-01 19:47:47.670647: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Num devices available:  [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Selected port: 37647


2024-11-01 19:47:50.179633: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered

NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow/tensorboard/issues/4784

Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.10.1 at http://localhost:37647/ (Press CTRL+C to quit)


In [2]:
ENV = "Ant-v2"
SUCESS_CRITERIA_VALUE = ENV_DETAILS[ENV]["SUCESS_CRITERIA_VALUE"]
SUCESS_CRITERIA_EPOCH = ENV_DETAILS[ENV]["SUCESS_CRITERIA_EPOCH"]
EPISODES = ENV_DETAILS[ENV]["EPISODES"]

In [3]:

env = gym.make(ENV)
env


<TimeLimit<OrderEnforcing<AntEnv<Ant-v2>>>>

In [4]:
s = env.observation_space.sample()
s

array([ 1.08792531, -0.29635189, -0.44762209,  0.50426208,  0.59140099,
       -0.11150847,  0.03198041, -0.86277575,  0.31130517,  0.4970839 ,
        0.84836422, -0.09278206,  1.35520174, -1.2353033 ,  0.7015194 ,
        1.46390142,  0.6725944 , -0.49279252, -0.50244316, -0.95914696,
        2.00422543,  1.16276881,  0.47958527,  0.07371542, -0.85467976,
       -0.81677467, -1.13175461,  0.29238508,  0.5169921 , -0.39676577,
       -0.52049487,  1.63481387, -0.34365855, -0.08577749, -1.10731558,
        0.17539998, -0.70575745, -0.33006325,  1.88751031, -0.88429754,
        0.23244635, -2.53789871, -0.90742981,  0.69837193, -1.68891222,
        0.98638459, -0.06750871,  0.47772436,  2.07411392, -0.83500376,
        2.12604605, -1.3351905 , -2.09651741, -0.24200982,  1.85534515,
       -0.31354958,  0.58229347, -1.81749607, -0.51276247,  0.67659263,
        0.11723784, -0.07826109,  0.22468299,  0.12211615,  1.14182122,
        0.73504896, -0.10520361, -1.11568192, -0.74955201, -0.12

In [5]:
env.observation_space, env.observation_space.shape

(Box(-inf, inf, (111,), float64), (111,))

In [6]:
s = env.reset()
s

array([ 0.68937016,  0.99335686, -0.06590261, -0.06598153, -0.06741987,
       -0.06225599, -0.02283051, -0.00647859,  0.02235212,  0.06714531,
       -0.0886551 ,  0.09654004, -0.08430423,  0.00604475, -0.05113899,
       -0.0135994 , -0.05960314, -0.0049479 ,  0.01363203,  0.21761968,
       -0.03197016,  0.10740787,  0.2243877 ,  0.0228236 , -0.0334111 ,
       -0.01556218, -0.0507538 ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.  

In [7]:
s = env.reset()
env.action_space, env.action_space

(Box(-1.0, 1.0, (8,), float32), Box(-1.0, 1.0, (8,), float32))

### Training/Hyperparam run

In [8]:
TUNING_TYPE = "BAYES"
HYPERPARAM_TUNING = True
writer= "Training/fit_PPO/"

## Select tunning parameters range

In [9]:
# if HYPERPARAM_TUNING: 

#     dir = r"Hyperparam_range"
#     project_name = "Random_search"

#     tuner = kt.RandomSearch(
#             MyHyperModel( hyper_dir = dir+"/"+project_name,  writer = "logs_hyper/ppo_hyperparameter_range/", evaluation_epoch = env._max_episode_steps, training_steps = 600000,
#                 sucess_criteria_epochs = SUCESS_CRITERIA_EPOCH, sucess_criteria_value= SUCESS_CRITERIA_VALUE,
#                 discount_min = 0.90, discount_max = 0.99, 
#                 gae_min = 0.85, gae_max = 0.96, 
#                 policy_clip =0.2,
#                 lr_actor_crit_min = 0.00001, lr_actor_crit_max = 0.001,
#                 #entropy_factor = 0.05,  # WAS THIS
#                 entropy_factor_min = 0.001, entropy_factor_max = 0.5,
#                 lr_model_min = 0.000001, lr_model_max =  0.001, kl_divergence_target = None,
#                 #dense_layers = [42,62],
#                 dense_min = 32, dense_max = 256, 
#                 environment_name=ENV, num_layers_act = 2, 
#                 num_layers_model = 1,
#                 training_epoch = 1,
#                 memory_size = env._max_episode_steps, 
#                 normalize_reward=False, normalize_advantage= True,
#                 scaling_factor_reward = 0.1
#                 #memory_size_max= env._max_episode_steps
#                 ),
#             objective= kt.Objective('total_train_reward', direction="max"), 
#             max_trials=10, 
#             directory=dir,
#             project_name=project_name
#         )
#     tuner.search(x=[0], y=[1])

#     for trials in tuner.oracle.get_best_trials(num_trials=3):
#         print(f"Trial id :{trials.trial_id} | Score :{trials.score} -->",trials.hyperparameters.values)

## Applying tunning range

In [10]:
if HYPERPARAM_TUNING: 

    dir = r"Hyperparam_kt_ppo"
    project_name = "keras_tunning_ppo_env_model_deep_dive"

    tuner = kt.BayesianOptimization(
            MyHyperModel( hyper_dir = dir+"/"+project_name,  writer = "logs_hyper/ppo_deep_dive/", evaluation_epoch = env._max_episode_steps, training_steps = 800000,
                sucess_criteria_epochs = SUCESS_CRITERIA_EPOCH, sucess_criteria_value= SUCESS_CRITERIA_VALUE,
                discount_min = 0.90, discount_max = 0.94, 
                #discount= 0.9,
                gae_factor = 0.89, 
                #gae_min = 0.89, gae_max = 0.90, 
                policy_clip =0.2,
                lr_actor_crit_min = 0.0002, lr_actor_crit_max = 0.001,
                entropy_factor = 0.015, 
                #entropy_factor_min = 0.01, entropy_factor_max = 0.03,
                lr_model_min = 0.0002, lr_model_max =  0.0007, kl_divergence_target = 0.17,
                dense_min = 128, dense_max = 256,
                environment_name=ENV, num_layers_act = 2, #max_num_layers_act = 2
                num_layers_model = 1, training_epoch = 1,
                memory_size = env._max_episode_steps, 
                normalize_reward=False, normalize_advantage= True,
                scaling_factor_reward = 0.1
                ),
            objective= kt.Objective('total_train_reward', direction="max"), 
            max_trials = 10,
            # distribution_strategy= strategy,
            directory=dir,
            project_name=project_name,
            #seed=0
        )
    tuner.search(x=[0], y=[1])
else : 
    
        print("Acquiring parameters ....")

        dense_units_act_crit = [128,64]
        num_layers_actor_critic = len(dense_units_act_crit)
        num_layer_m = 1
        dense_units_model = [12]

        model = run_training(training_steps = 500000,
                            discount = 0.99, 
                            dense_units_act_crit = dense_units_act_crit,  
                            dense_units_model = dense_units_model,  
                            num_layer_a_c = num_layers_actor_critic,  
                            num_layer_m = num_layer_m, 
                            writer = writer, 
                            environment_name = ENV, 
                            return_agent = True, 
                            lr_actor_critic= 0.0001,  
                            lr_model = 0.01,
                            sucess_criteria_epochs=SUCESS_CRITERIA_EPOCH, 
                            sucess_criteria_value = SUCESS_CRITERIA_VALUE, 
                            gae_lambda = 0.95, 
                            entropy_coeff = 0.05, 
                            policy_clip = 0.2, training_epoch = 20, 
                            scaling_factor_reward = 0.1, 
                            kl_divergence_target = 0.01,
                            memory_size =  env._max_episode_steps,
                            normalize_reward=False, normalize_advantage= False, use_mlflow = False)


Reloading Tuner from Hyperparam_kt_ppo/keras_tunning_ppo_env_model_deep_dive/tuner0.json


In [11]:
for trials in tuner.oracle.get_best_trials(num_trials=10):
    print(f"Trial id :{trials.trial_id} | Score :{trials.score} -->",trials.hyperparameters.values)

Trial id :04 | Score :433.7122055783411 --> {'discount': 0.9, 'lr_actor_critic': 0.0005134394132368825, 'lr_model': 0.0003268575286591344, 'dense_units_act_crit_0': 220, 'dense_units_act_crit_1': 198, 'n_dense_layers_model0': 219}
Trial id :07 | Score :322.04213972786926 --> {'discount': 0.91, 'lr_actor_critic': 0.0003529097867523926, 'lr_model': 0.0006339329907047511, 'dense_units_act_crit_0': 147, 'dense_units_act_crit_1': 205, 'n_dense_layers_model0': 234}
Trial id :09 | Score :306.31369484503904 --> {'discount': 0.91, 'lr_actor_critic': 0.00044957085455382803, 'lr_model': 0.0003458313661277884, 'dense_units_act_crit_0': 131, 'dense_units_act_crit_1': 188, 'n_dense_layers_model0': 154}
Trial id :02 | Score :250.43993040486293 --> {'discount': 0.92, 'lr_actor_critic': 0.00028712352969576015, 'lr_model': 0.0005988145071461011, 'dense_units_act_crit_0': 212, 'dense_units_act_crit_1': 246, 'n_dense_layers_model0': 250}
Trial id :00 | Score :222.71912150699106 --> {'discount': 0.9, 'lr_a

In [12]:
# actorcritic_agent = tuner.get_best_models(num_models = 1)[0]
# actorcritic_agent.training_steps = 2000000
# actorcritic_agent.writer =  "logs_hyper/ppo_training/"
# actorcritic_agent.train_agent(monitoring_epoch = 10000)
# final_rewards = final_evaluation(actorcritic_agent, val_env,n_tries=200, exploration=exploration_tech,  video_name = "./ppo_"+exploration_tech+"_video.mp4", sucess_criteria_epochs= SUCESS_CRITERIA_EPOCH)
# results.append(final_rewards)
# print("Final mean reward '", final_rewards)


In [13]:
actorcritic_agent = tuner.get_best_models(num_models = 2)[1]
actorcritic_agent.training_steps = 2000000
actorcritic_agent.writer =  "logs_hyper/ppo_training/"
actorcritic_agent.train_agent(monitoring_epoch = 10000)
final_rewards = final_evaluation(actorcritic_agent, val_env,n_tries=200, exploration=exploration_tech,  video_name = "./ppo_"+exploration_tech+"_video_v2.mp4", sucess_criteria_epochs= SUCESS_CRITERIA_EPOCH)
results.append(final_rewards)
print("Final mean reward '", final_rewards)


In [14]:
# actorcritic_agent = tuner.get_best_models(num_models = 1)[0]
# actorcritic_agent.training_steps = 2000000
# actorcritic_agent.writer =  "logs_hyper/ppo_training/"
# actorcritic_agent.train_agent(monitoring_epoch = 10000)
# final_rewards = final_evaluation(actorcritic_agent, val_env,n_tries=200, exploration=exploration_tech,  video_name = "./ppo_"+exploration_tech+"_video.mp4", sucess_criteria_epochs= SUCESS_CRITERIA_EPOCH)
# results.append(final_rewards)
# print("Final mean reward '", final_rewards)


In [ ]:
val_env = gym.make(ENV) aaaaaa
exploration_tech = "soft"
hyperparam_combination=[]

#results = []
for best_hps in tuner.get_best_hyperparameters(num_trials=1):
    actorcritic_agent = tuner.hypermodel.build(best_hps)  # Build the model with best hyperparameters
    #actorcritic_agent = rerun_training(training_steps  = 2000000, model = env_model)

# for trials in tuner.oracle.get_best_trials(num_trials=1):
#     print(f"Trial id :{trials.trial_id} | Score :{trials.score} -->",trials.hyperparameters.values)

#     discount = trials.hyperparameters.values["discount"]
#     gae_lambda = 0.89
#     lr_actor_critic = trials.hyperparameters.values['lr_actor_critic']
#     lr_model = trials.hyperparameters.values['lr_model']
#     policy_clip =0.2
#     scaling_factor_reward = 0.1
#     entropy_coeff = 0.015

#     num_layer_act_crit = 2
#     dense_units_act_crit =  [trials.hyperparameters.values['dense_units_act_crit_'+str(i)] for i in range(num_layer_act_crit)]

#     num_layer_m = 1
#     dense_units_model =  [trials.hyperparameters.values['n_dense_layers_model'+str(i)] for i in range(num_layer_m)]

#     actorcritic_agent = PPO(
#         training_steps = 2000000, 
#         sucess_criteria_epochs  = SUCESS_CRITERIA_EPOCH, 
#         sucess_criteria_value = SUCESS_CRITERIA_VALUE,
#         discount = discount, 
#         dense_units_act_crit = dense_units_act_crit,
#         dense_units_model = dense_units_model,
#         num_layer_act_crit  = num_layer_act_crit, 
#         num_layer_model = num_layer_m,
#         writer =  "logs_hyper/ppo_training/",
#         evaluation_epoch = env._max_episode_steps,
#         environment_name = ENV,
#         lr_actor_critic = lr_actor_critic,  lr_model = lr_model, 
#         gae_lambda=gae_lambda,
#         policy_clip = policy_clip,
#         training_epoch = 1,
#         entropy_coeff = entropy_coeff,
#         memory_size = env._max_episode_steps,
#         scaling_factor_reward = scaling_factor_reward,
#         normalize_reward = False,
#         normalize_advantage = True, 
#         kl_divergence_target = 0.17,
#         reward_norm_factor = 0.1,
#         force_extreme_exploration = False
#         )
    actorcritic_agent.training_steps = 2000000
    actorcritic_agent.writer =  "logs_hyper/ppo_training/"
    actorcritic_agent.train_agent(monitoring_epoch = 10000)

    final_rewards = final_evaluation(actorcritic_agent, val_env,n_tries=200, exploration=exploration_tech,  video_name = "./ppo_"+exploration_tech+"_video.mp4", sucess_criteria_epochs= SUCESS_CRITERIA_EPOCH)
    results.append(final_rewards)
    print("Final mean reward '", final_rewards)

Trial number :  10
Epoch: 2000 : Reward eval/Train: -86.84746123751128/-114.90911496950007 
Epoch: 4000 : Reward eval/Train: -60.50770143376253/-129.56236768860128 
Epoch: 6000 : Reward eval/Train: -70.36720529915142/-155.24393418812687 


In [ ]:
# project_name = "keras_tunning_ppo_env_model_deep_dive"

# tuner = kt.BayesianOptimization(
#         MyHyperModel( hyper_dir = dir+"/"+project_name,  writer = "logs_hyper/ppo_deep_dive/", evaluation_epoch = env._max_episode_steps, training_steps = 1000000,
#             sucess_criteria_epochs = SUCESS_CRITERIA_EPOCH, sucess_criteria_value= SUCESS_CRITERIA_VALUE,
#             discount_min = 0.90, discount_max = 0.94, 
#             #discount= 0.99,
#             #gae_factor = 0.95, 
#             gae_min = 0.88, gae_max = 0.90, 
#             policy_clip =0.2,
#             lr_actor_crit_min = 0.0002, lr_actor_crit_max = 0.001,
#             #entropy_factor = 0.05, 
#             entropy_factor_min = 0.01, entropy_factor_max = 0.03,
#             lr_model_min = 0.0002, lr_model_max =  0.0007, kl_divergence_target = 0.17,
#             #dense_layers = [128,128],
#             dense_min = 128, dense_max = 256,
#             environment_name=ENV, num_layers_act = 2, #max_num_layers_act = 2
#             num_layers_model = 1, training_epoch = 1,
#             memory_size = env._max_episode_steps, 
#             normalize_reward=False, normalize_advantage= True,
#             scaling_factor_reward = 0.1
#             #memory_size_max= env._max_episode_steps
#             ),
#         objective= kt.Objective('total_train_reward', direction="max"), 
#         max_trials = 10,
#         # distribution_strategy= strategy,
#         directory=dir,
#         project_name=project_name,
#         #seed=0
#     )
# #tuner.search(x=[0], y=[1])

